In [9]:
#Optimizacion Bayesiana de hiperparametros de  rpart

#limpio la memoria
rm( list=ls() )  #remove all objects
gc()             #garbage collection

require("data.table")
require("rlist")

require("rpart")
require("parallel")

#paquetes necesarios para la Bayesian Optimization
require("DiceKriging")
require("mlrMBO")


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2121198,113.3,4348665,232.3,2597854,138.8
Vcells,3564931,27.2,28968204,221.1,70619090,538.8


In [10]:
#Defino la  Optimizacion Bayesiana

kBO_iter  <- 100   #cantidad de iteraciones de la Optimizacion Bayesiana

hs  <- makeParamSet(
          makeNumericParam("cp"       , lower= -1   , upper=    0.1),
          makeIntegerParam("minsplit" , lower=  1L  , upper= 8000L),  #la letra L al final significa ENTERO
          makeIntegerParam("minbucket", lower=  1L  , upper= 4000L),
          makeIntegerParam("maxdepth" , lower=  3L  , upper=   20L),
          forbidden = quote( minbucket > 0.5*minsplit ) )             # minbuket NO PUEDE ser mayor que la mitad de minsplit

ksemilla_azar  <- 124739   #cambiar por la primer semilla

In [11]:
#graba a un archivo los componentes de lista
#para el primer registro, escribe antes los titulos

loguear  <- function( reg, arch=NA, folder="./work/", ext=".txt", verbose=TRUE )
{
  archivo  <- arch
  if( is.na(arch) )  archivo  <- paste0( folder, substitute( reg), ext )

  if( !file.exists( archivo ) )  #Escribo los titulos
  {
    linea  <- paste0( "fecha\t", 
                      paste( list.names(reg), collapse="\t" ), "\n" )

    cat( linea, file=archivo )
  }

  linea  <- paste0( format(Sys.time(), "%Y%m%d %H%M%S"),  "\t",     #la fecha y hora
                    gsub( ", ", "\t", toString( reg ) ),  "\n" )

  cat( linea, file=archivo, append=TRUE )  #grabo al archivo

  if( verbose )  cat( linea )   #imprimo por pantalla
}

In [12]:
#particionar agrega una columna llamada fold a un dataset que consiste en una particion estratificada segun agrupa
# particionar( data=dataset, division=c(70,30), agrupa=clase_ternaria, seed=semilla)   crea una particion 70, 30 
# particionar( data=dataset, division=c(1,1,1,1,1), agrupa=clase_ternaria, seed=semilla)   divide el dataset en 5 particiones

particionar  <- function( data, division, agrupa="", campo="fold", start=1, seed=NA )
{
  if( !is.na( seed)  )   set.seed( seed )

  bloque  <- unlist( mapply(  function(x,y) { rep( y, x ) }, division, seq( from=start, length.out=length(division) )  ) )

  data[ , (campo) :=  sample( rep( bloque, ceiling(.N/length(bloque))) )[1:.N],
           by= agrupa ]
}

In [13]:
#fold_test  tiene el numero de fold que voy a usar para testear, entreno en el resto de los folds
#param tiene los hiperparametros del arbol

ArbolSimple  <- function( fold_test, data, param )
{
  #genero el modelo
  modelo  <- rpart("clase_ternaria ~ .", 
                   data= data[ fold != fold_test, ],  #entreno en todo MENOS el fold_test que uso para testing
                   xval= 0,
                   control= param )

  #aplico el modelo a los datos de testing
  prediccion  <- predict( modelo, 
                          data[ fold==fold_test, ],  #aplico el modelo sobre los datos de testing
                          type= "prob")   #quiero que me devuelva probabilidades

  prob_baja2  <- prediccion[, "BAJA+2"]  #esta es la probabilidad de baja

  #calculo la ganancia
  ganancia_testing  <- data[ fold==fold_test ][ prob_baja2 > 1/40,  
                                                sum( ifelse( clase_ternaria=="BAJA+2", 78000, -2000 ) )] 

  return( ganancia_testing )  #esta es la ganancia sobre el fold de testing, NO esta normalizada
}

In [14]:
ArbolesCrossValidation  <- function( data, param, qfolds, pagrupa, semilla )
{
  divi  <- rep( 1, qfolds )  # generalmente  c(1, 1, 1, 1, 1 )  cinco unos

  particionar( data, divi, seed=semilla, agrupa=pagrupa )  #particiono en dataset en folds

  ganancias  <- mcmapply( ArbolSimple, 
                          seq(qfolds), # 1 2 3 4 5
                          MoreArgs= list( data, param), 
                          SIMPLIFY= FALSE,
                          mc.cores= 1 )   #se puede subir a qfolds si posee Linux o Mac OS

  data[ , fold := NULL ]

  #devuelvo la primer ganancia y el promedio
  ganancia_promedio  <- mean( unlist( ganancias ) )   #promedio las ganancias
  ganancia_promedio_normalizada  <- ganancia_promedio * qfolds  #aqui normalizo la ganancia
  
  return( ganancia_promedio_normalizada )
}

In [15]:
#esta funcion solo puede recibir los parametros que se estan optimizando
#el resto de los parametros, lamentablemente se pasan como variables globales

EstimarGanancia  <- function( x )
{
   GLOBAL_iteracion  <<-  GLOBAL_iteracion + 1

   xval_folds  <- 5
   ganancia  <- ArbolesCrossValidation( dataset,
                                        param= x, #los hiperparametros del arbol
                                        qfolds= xval_folds,  #la cantidad de folds
                                        pagrupa= "clase_ternaria",
                                        semilla= ksemilla_azar )

   #logueo 
   xx  <- x
   xx$xval_folds  <-  xval_folds
   xx$ganancia  <- ganancia
   xx$iteracion <- GLOBAL_iteracion
   loguear( xx,  arch= archivo_log )

   return( ganancia )
}

In [16]:
#Aqui empieza el programa

setwd( "C:\\Users\\jesia\\Desktop\\4_DMEyF\\" )

#cargo el dataset
dataset  <- fread("./datasets/competencia1_2022.csv")   #donde entreno


#creo la carpeta donde va el experimento
# HT  representa  Hiperparameter Tuning
dir.create( "./exp/",  showWarnings = FALSE ) 
dir.create( "./exp/HT3210/", showWarnings = FALSE )
setwd("./exp/HT3210/")   #Establezco el Working Directory DEL EXPERIMENTO


archivo_log  <- "HT321.txt"
archivo_BO   <- "HT321.RDATA"

#leo si ya existe el log, para retomar en caso que se se corte el programa
GLOBAL_iteracion  <- 0

if( file.exists(archivo_log) )
{
 tabla_log  <- fread( archivo_log )
 GLOBAL_iteracion  <- nrow( tabla_log )
}



In [17]:
#Aqui comienza la configuracion de la Bayesian Optimization

funcion_optimizar  <- EstimarGanancia

configureMlr( show.learner.output= FALSE)

#configuro la busqueda bayesiana,  los hiperparametros que se van a optimizar
#por favor, no desesperarse por lo complejo
obj.fun  <- makeSingleObjectiveFunction(
              fn=       funcion_optimizar,
              minimize= FALSE,   #estoy Maximizando la ganancia
              noisy=    TRUE,
              par.set=  hs,
              has.simple.signature = FALSE
             )

ctrl  <- makeMBOControl( save.on.disk.at.time= 600,  save.file.path= archivo_BO)
ctrl  <- setMBOControlTermination(ctrl, iters= kBO_iter )
ctrl  <- setMBOControlInfill(ctrl, crit= makeMBOInfillCritEI())

surr.km  <- makeLearner("regr.km", predict.type= "se", covtype= "matern3_2", control= list(trace= TRUE))

#inicio la optimizacion bayesiana
if( !file.exists( archivo_BO ) ) {

  run  <- mbo( fun=     obj.fun, 
               learner= surr.km,
               control= ctrl)

} else  run  <- mboContinue( archivo_BO )   #retomo en caso que ya exista



Computing y column(s) for design. Not provided.



20220910 195550	-0.663718881786917	4402	1964	9	5	17016000	1
20220910 195651	-0.854227190825622	3423	1666	19	5	17590000	2
20220910 195751	-0.555623859382467	6435	934	14	5	16976000	3
20220910 195841	-0.049588153474906	5931	79	9	5	16576000	4
20220910 195930	-0.863048658902699	6996	3336	13	5	15802000	5
20220910 200003	-0.435702209998271	7649	2699	4	5	16200000	6
20220910 200103	-0.34933815109689	5135	446	20	5	17016000	7
20220910 200139	-0.738925550576823	2364	1053	5	5	17714000	8
20220910 200229	-0.618308201679611	7500	1566	11	5	16228000	9
20220910 200307	-0.320804992070771	6753	1069	6	5	16228000	10
20220910 200403	-0.254482050295337	4268	402	14	5	17266000	11
20220910 200451	-0.935745532158762	6945	2866	20	5	16244000	12
20220910 200533	-0.54923052824568	4052	536	8	5	17266000	13
20220910 200627	-0.382657586938391	6316	1137	13	5	17016000	14
20220910 200702	-0.189319560083095	6955	145	5	5	16142000	15
20220910 200817	-0.99551648828201	3075	99	19	5	16834000	16


[mbo] 0: cp=-0.664; minsplit=4402; minbucket=1964; maxdepth=9 : y = 1.7e+07 : 48.5 secs : initdesign

[mbo] 0: cp=-0.854; minsplit=3423; minbucket=1666; maxdepth=19 : y = 1.76e+07 : 60.8 secs : initdesign

[mbo] 0: cp=-0.556; minsplit=6435; minbucket=934; maxdepth=14 : y = 1.7e+07 : 60.3 secs : initdesign

[mbo] 0: cp=-0.0496; minsplit=5931; minbucket=79; maxdepth=9 : y = 1.66e+07 : 49.8 secs : initdesign

[mbo] 0: cp=-0.863; minsplit=6996; minbucket=3336; maxdepth=13 : y = 1.58e+07 : 49.2 secs : initdesign

[mbo] 0: cp=-0.436; minsplit=7649; minbucket=2699; maxdepth=4 : y = 1.62e+07 : 32.6 secs : initdesign

[mbo] 0: cp=-0.349; minsplit=5135; minbucket=446; maxdepth=20 : y = 1.7e+07 : 60.5 secs : initdesign

[mbo] 0: cp=-0.739; minsplit=2364; minbucket=1053; maxdepth=5 : y = 1.77e+07 : 35.2 secs : initdesign

[mbo] 0: cp=-0.618; minsplit=7500; minbucket=1566; maxdepth=11 : y = 1.62e+07 : 50.0 secs : initdesign

[mbo] 0: cp=-0.321; minsplit=6753; minbucket=1069; maxdepth=6 : y = 1.62e+

20220910 200915	-0.458810071812163	2544	1231	13	5	17030000	17


[mbo] 1: cp=-0.459; minsplit=2544; minbucket=1231; maxdepth=13 : y = 1.7e+07 : 55.7 secs : infill_ei



...

[mbo] 98: cp=-0.946; minsplit=1928; minbucket=207; maxdepth=9 : y = 1.99e+07 : 51.7 secs : infill_ei



20220910 214533	-0.546261507096332	4355	235	20	5	16878000	115


[mbo] 99: cp=-0.546; minsplit=4355; minbucket=235; maxdepth=20 : y = 1.69e+07 : 60.2 secs : infill_ei



20220910 214637	-0.91867234121504	1663	225	19	5	20356000	116


[mbo] 100: cp=-0.919; minsplit=1663; minbucket=225; maxdepth=19 : y = 2.04e+07 : 62.7 secs : infill_ei

Saved the final state in the file HT321.RDATA

